In [1]:
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Activation, Input

2024-10-29 23:49:15.525140: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730267355.536212  207074 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730267355.539539  207074 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-29 23:49:15.550674: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
FILE = "./db/plan_de_ayala.txt"

In [3]:
# Extract the text
print("Extracting text from input...")
with open(FILE, 'rb') as fin:
    lines = [line.decode("utf-8").strip().lower() for line in fin if line.strip()]

text = " ".join(lines)
print(text)

Extracting text from input...
plan de ayala. plan libertador de los hijos del estado de morelos afiliados al ejército insurgente que defiende el cumplimiento del plan de s. luis, con las reformas que ha creido conveniente aumentar en beneficio de la patria mexicana. los que subscribimos, constituidos en junta revolucionaria, para sostener y llevar a cabo las promesas que hizo la revolución de 20 de noviembre de 1910, próximo pasado, declaramos solemnemente ante la faz del mundo civilizado, que nos juzga, y ante la nación a que pertenecemos y amamos, los propósitos que hemos formulado para acabar con la tiranía que nos oprime y redimir a la patria de las dictaduras que nos imponen, las cuales quedan determinadas en el siguiente plan. 1.° teniendo en consideración que el pueblo mexicano acaudillado por don francisco i. madero fué a derramar su sangre para conquistar sus libertades y reivindicar sus derechos conculcados, y no para que un hombre se adueñara del poder violando los sagrados 

In [4]:
# Create character mappings
chars = sorted(set(text))
nb_chars = len(chars)
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

In [5]:
# Create input and label data from text
print("Creating input and label text...")
SEQLEN = 10
STEP = 1
num_sequences = (len(text) - SEQLEN) // STEP + 1

Creating input and label text...


In [6]:
# Allocate output matrices
print("Vectorizing input and label text...")
X = np.zeros((num_sequences, SEQLEN, nb_chars), dtype=int)
y = np.zeros((num_sequences, nb_chars), dtype=int)

# Vectorization
for i in range(num_sequences):
    start_idx = i * STEP
    input_char = text[start_idx:start_idx + SEQLEN]
    X[i, np.arange(SEQLEN), [char2index[ch] for ch in input_char]] = 1

    if start_idx + SEQLEN < len(text):
        label_char = text[start_idx + SEQLEN]
        y[i, char2index[label_char]] = 1

Vectorizing input and label text...


In [7]:
# Display shapes of resulting vectors
print(X.shape)
print(y.shape)

(11888, 10, 52)
(11888, 52)


In [8]:
# Verify character encoding
print(X[0])
print(y[0])

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0

In [9]:
# Build the model
HIDDEN_SIZE = 128
BATCH_SIZE = 128
NUM_ITERATIONS = 25
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

model = Sequential()
model.add(Input(shape=(SEQLEN, nb_chars)))
model.add(SimpleRNN(HIDDEN_SIZE, return_sequences=False, unroll=True))
model.add(Dense(nb_chars))
model.add(Activation("softmax"))


model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

2024-10-29 23:49:37.182331: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-10-29 23:49:37.182366: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:137] retrieving CUDA diagnostic information for host: andres-b460mds3h
2024-10-29 23:49:37.182370: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:144] hostname: andres-b460mds3h
2024-10-29 23:49:37.182413: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:168] libcuda reported version is: 550.120.0
2024-10-29 23:49:37.182425: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:172] kernel reported version is: 550.120.0
2024-10-29 23:49:37.182428: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:259] kernel version seems to match DSO: 550.120.0


In [10]:
# Train the model in batches
for iteration in range(NUM_ITERATIONS):
    print("=" * 50)
    print("Iteration #: %d" % iteration)
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Testing model
    test_idx = np.random.randint(len(X))
    test_chars = text[test_idx:test_idx + SEQLEN]
    print("Generating from seed: %s" % test_chars)
    print(test_chars, end="")

    for _ in range(NUM_PREDS_PER_EPOCH):
        Xtest = np.zeros((1, SEQLEN, nb_chars), dtype=int)
        for i, ch in enumerate(test_chars):
            Xtest[0, i, char2index[ch]] = 1
        
        pred = model.predict(Xtest)[0]
        ypred = index2char[np.argmax(pred)]
        print(ypred, end="")

        # Move forward with test_chard + ypred
        test_chars = test_chars[1:] + ypred
    print()

Iteration #: 0
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 3.1052
Generating from seed: s títulos 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━

KeyboardInterrupt: 